In [1]:
import pandas as pd
import os
import time
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike

# ================= 配置区域 =================
MY_API_KEY = "sk-bf54fbae108748da859244787c7445d0" 

llm = OpenAILike(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key=MY_API_KEY,
    is_chat_model=True,
    temperature=0
)
Settings.llm = llm

def run_challenge_baseline():
    print("\n🚀 开始运行 Baseline 对照组 (Adversarial Challenge Set)...")
    
    # 定义之前的 3 个陷阱任务
    tasks = [
        {
            "file": "bench_currency.csv",
            "question": "请计算 amount 列的平均值",
            "name": "Challenge 1: Currency Symbols"
        },
        {
            "file": "bench_dates.csv",
            "question": "请告诉我 transaction_id 103 是在哪个月份 (month) 发生的？(请提取月份数字)",
            "name": "Challenge 2: Mixed Date Formats"
        },
        {
            "file": "bench_units.csv",
            "question": "请计算 weight_val 列的总和 (sum)",
            "name": "Challenge 3: Mixed Units"
        }
    ]
    
    results = []

    for task in tasks:
        print(f"正在执行: {task['name']} ...")
        
        if not os.path.exists(task['file']):
            print(f"❌ 找不到 {task['file']}，请先运行 generate_benchmark.py")
            continue
            
        try:
            df = pd.read_csv(task['file'])
            # 没有任何纠错机制的 Baseline
            engine = PandasQueryEngine(df=df, verbose=False)
            
            start_time = time.time()
            response = engine.query(task['question'])
            final_output = str(response)
            
            # 严格检查：如果有 Error 或者结果显然不对（比如简单的字符串拼接）
            if "Error" in final_output or "Crash" in final_output:
                status = "❌ CRASH"
            else:
                # 特殊检查：对于 amount 求平均，如果返回的是字符串拼接（如 '10002000...'），也是失败
                # 对于 Baseline 来说，通常只要不报错就算 Success，但我们要看看内容
                status = "✅ SUCCESS (Maybe)"
                
            print(f"    状态: {status}")
            print(f"    输出: {final_output[:50]}...")
            
            results.append({"Task": task['name'], "Status": status})
            
        except Exception as e:
            print(f"    ❌ 运行崩溃: {type(e).__name__}")
            results.append({"Task": task['name'], "Status": "❌ CRASH"})

    print("\n" + "="*50)
    success_count = len([r for r in results if "SUCCESS" in r["Status"]])
    print(f"🏆 Baseline 在挑战集上的成功率: {success_count}/{len(results)}")
    print("="*50)

if __name__ == "__main__":
    run_challenge_baseline()


🚀 开始运行 Baseline 对照组 (Adversarial Challenge Set)...
正在执行: Challenge 1: Currency Symbols ...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


<unknown>:1: SyntaxWarning: invalid escape sequence '\$'


    状态: ✅ SUCCESS (Maybe)
    输出: 2000.0...
正在执行: Challenge 2: Mixed Date Formats ...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
    状态: ✅ SUCCESS (Maybe)
    输出: March 5, 2023...
正在执行: Challenge 3: Mixed Units ...
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
    状态: ✅ SUCCESS (Maybe)
    输出: 180.0...

🏆 Baseline 在挑战集上的成功率: 3/3
